- DistilBERT를 이용한 감성분석 모델
- KoBERT 모델과 모든 테스크 동일
-  

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
import sys

print(os.getcwd())

/content


In [5]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

ERROR: Operation cancelled by user


In [4]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/dndhma/KoBERT.git@master

  Cloning https://****@github.com/dndhma/KoBERT.git (to revision master) to /tmp/pip-req-build-b2j7pq0v
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/dndhma/KoBERT.git' /tmp/pip-req-build-b2j7pq0v
  Resolved https://****@github.com/dndhma/KoBERT.git to commit bfb47f37c6aab46bf11a798690c8c66faf82110d
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 74.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 91.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 10.

# Data Preparation and Training

In [6]:
!pip install tokenizers
import numpy as np
np.bool = np.bool_
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook

In [7]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [8]:
#GPU 사용
device = torch.device("cuda")

In [9]:
from transformers import BertModel, DistilBertModel
bert_model = BertModel.from_pretrained('monologg/kobert')
distilbert_model = DistilBertModel.from_pretrained('monologg/distilkobert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [11]:
import pandas as pd
chatbot_data = pd.read_csv('sentiment_dialogues.csv')

In [12]:
chatbot_data.loc[(chatbot_data[ '감정'] == "불안"),  '감정'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data[ '감정'] == "당황"),  '감정'] = 1  #당황 => 1
chatbot_data.loc[(chatbot_data[ '감정'] == "분노"),  '감정'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data[ '감정'] == "슬픔"),  '감정'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data[ '감정'] == "중립"),  '감정'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data[ '감정'] == "행복"),  '감정'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data[ '감정'] == "혐오"),  '감정'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['발화'], chatbot_data[ '감정'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0)

In [13]:
print(len(dataset_train))
print(len(dataset_test))

127513
19054


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))


    def __len__(self):
        return (len(self.labels))

In [16]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [17]:
# 토큰화
tokenizer = get_tokenizer()

# tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [19]:
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length,why):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        pooler = self.bert(input_ids = token_ids, attention_mask = attention_mask.float().to(token_ids.device))[0][:,0]
        # distilbert는 pooler를 return하지 않음. 따라서 last_hidden_state로 부터 직접 pooler를 추출함

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

model = BERTClassifier(distilbert_model,  dr_rate=0.5).to(device)


In [20]:
#BERT 모델 불러오기
# model = BERTClassifier(distilbert_model,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

# train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/drive/MyDrive/" #your path
ckpt_name=ckpt_path+"saved_model_distilbert.pt"

In [21]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device) ## DistillBERT는 token_type_ids(segment_ids)를 사용하지 않음.
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        # segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
          out = model(token_ids, valid_length)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc

      print('current best model saved')


<ipython-input-21-0b7efc5f2966>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1993 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.967143177986145 train acc 0.15625
epoch 1 batch id 201 loss 1.818773865699768 train acc 0.21517412935323382
epoch 1 batch id 401 loss 1.6350107192993164 train acc 0.2821072319201995
epoch 1 batch id 601 loss 1.4657026529312134 train acc 0.3301268718801997
epoch 1 batch id 801 loss 1.350101351737976 train acc 0.3651880461922597
epoch 1 batch id 1001 loss 1.3141084909439087 train acc 0.3943244255744256
epoch 1 batch id 1201 loss 1.1532049179077148 train acc 0.41694421315570357
epoch 1 batch id 1401 loss 1.232692003250122 train acc 0.4379907209136331
epoch 1 batch id 1601 loss 1.0694448947906494 train acc 0.4565017957526546
epoch 1 batch id 1801 loss 0.9706892371177673 train acc 0.475039908384231
epoch 1 train acc 0.4900743226292022


<ipython-input-21-0b7efc5f2966>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length,segment_ids,label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/298 [00:00<?, ?it/s]

...

epoch 14 test acc 0.6994273416982784 test loss 0.6980845332145691


  0%|          | 0/1993 [00:00<?, ?it/s]

epoch 15 batch id 1 loss 0.533343493938446 train acc 0.765625
epoch 15 batch id 201 loss 0.318990558385849 train acc 0.8303016169154229
epoch 15 batch id 401 loss 0.543247401714325 train acc 0.8282029301745636
epoch 15 batch id 601 loss 0.5195703506469727 train acc 0.8283069883527454
epoch 15 batch id 801 loss 0.4363821744918823 train acc 0.8278323970037453
epoch 15 batch id 1001 loss 0.6905080080032349 train acc 0.8275474525474525
epoch 15 batch id 1201 loss 0.3018050193786621 train acc 0.8269671107410491
epoch 15 batch id 1401 loss 0.6940087080001831 train acc 0.8276342790863669
epoch 15 batch id 1601 loss 0.45646747946739197 train acc 0.827432073703935
epoch 15 batch id 1801 loss 0.49966710805892944 train acc 0.8287323014991671
epoch 15 train acc 0.8287547039638736


  0%|          | 0/298 [00:00<?, ?it/s]

epoch 15 test acc 0.6985884155237818 test loss 0.6960588693618774


# Inference

In [22]:
#새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model.eval()
    with torch.no_grad():

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)


          test_eval=[]
          for i in out:
              logits=i
              logits = logits.detach().cpu().numpy()


              if np.argmax(logits) == 0:
                  return "불안"
              elif np.argmax(logits) == 1:
                  return "당황"
              elif np.argmax(logits) == 2:
                  return "분노"
              elif np.argmax(logits) == 3:
                  return "슬픔"
              elif np.argmax(logits) == 4:
                  return "중립"
              elif np.argmax(logits) == 5:
                  return "행복"
              elif np.argmax(logits) == 6:
                  return "혐오"
              return logits
predict('하이')

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


'당황'

In [23]:
checkpoint=torch.load(ckpt_name)

In [24]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
import json

"""
format: [{"title":"[💙자유]단대소고 설명회 갔는데요...","content":"강당 자리를 사람들이 거의 다 채워서 1번 놀라고\n그 앞에서 에어컨 자랑, 급식 자랑하시는 교장님을 보고 2번 놀랐습니다\n교장님 귀여우시네요\n","comments":["어우 쒯 ㅋㅋ","학교 자랑한답시고 하는게 “에어컨”, “급식” ㅋㅋㅋㅋㅋㅋ","급식으로 매일 짜장면 시켜드시는  교직원분도 계신디...쓰읍","???:단대부고 화이팅","conalt=\"킹받네\""]},
"""

res = {}
# save data in format of


json_data = open('output.json', 'r', encoding='utf-8').read()
# Parse JSON data
data = json.loads(json_data)



# Iterate over all content and comments
for entry in data:
    # Process content
    if entry['content'] == '':
        continue
    res[entry['content']] = predict(entry['content'])
    print("predicted", res[entry['content']],"for", entry['content'])
    # Process comments
    if 'comments' not in entry:
        continue
    for comment in entry['comments']:

        if comment == '':
            continue
        res[comment] = predict(comment)
        print("predicted ", res[comment], "for", comment)

## save
with open('processed.json', 'w', encoding='utf-8') as f:
    json.dump(res, f, ensure_ascii=False, indent=4)

{'title': '[💙자유]단대소고 설명회 갔는데요...', 'content': '강당\xa0자리를 사람들이 거의 다 채워서\xa01번 놀라고\n그 앞에서 에어컨 자랑, 급식 자랑하시는 교장님을 보고 2번 놀랐습니다\n교장님 귀여우시네요\n', 'comments': ['어우 쒯 ㅋㅋ', '학교 자랑한답시고 하는게 “에어컨”, “급식” ㅋㅋㅋㅋㅋㅋ', '급식으로 매일 짜장면 시켜드시는  교직원분도 계신디...쓰읍', '???:단대부고 화이팅', 'conalt="킹받네"']}
{'title': '[💙자유]그립읍니다 오장원...', 'content': '안 만나봤긴 해ㅋ\n'}
{'title': '[💙자유]전교회장 선거 기권표 찍을거면 개추 ㅋㅋ', 'content': '재미도 없고\n감동도 없고\n\nㅋㅋㅋㅋㅋ\n\n', 'comments': ['대도혁']}
{'title': '[💙자유]2학년 5인큐', 'content': '럭스\n그라가스\n룰루\n세트\n다리우스\n', 'comments': ['이러네 ㅋㅋ']}
{'title': '[💙정보]인터넷 괴담이 무서워서 잠을 못 잘 때', 'content': '\n\n', 'comments': ['든든함은 오평 ㅋㅋ', '부매 넣어뒀으니 승인하셈']}
{'title': '[💙자유]권위에 도전하지 말라', 'content': '\n\n\n권위에 도전하지 말라\n\n1. 글 쓰는데는 자유가 있지만 그에 대한 책임은 본인이 져야 하는 것이 맞다.\n\n2. 갤러리 운영규칙에 따라 교사 언급은 교장을 제외하고는 언급을 하지 않았으면 좋겠다.\n\n3. 모 선생님의 글은 조치에 들어갈 예정이다.\n\n4. 최대한 글 삭제는 안하는 쪽으로 가는게 운영방향이지만 규칙에 따라 처리하는게 우선이다.\n\n5. 여러분들의 자유를 존중하나 남에게 피해를 주면서 쟁취하는 자유는 큰 결단이 필요하다. 이번 일을 계기로 잘 생각해보시라.\n\n6. 권위에 도전하지 말라. 완장질이 하고 싶으면 공식적인 창구를 통해 요청하라. 파딱 정도는 줄 

Exception ignored in: <function _releaseLock at 0x79afa5fea290>
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
